In [1]:
import os; os.chdir("/home/kabil/sietch/courses/bmes301/lab2")

In [2]:
%load_ext autoreload
%autoreload 2

## Import necessary libraries
### Data analysis
from scipy.signal import argrelextrema
from scipy import stats
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np

### Plotting
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns


### Setup
from pathlib import Path
import re

### Custom functions
import tools

## Define data path
datapath = Path("data")

In [62]:
# Load data

## Regular expression to pull team information from file name
meta = re.compile(r"g(?P<team>\d{2})")

## Load all data into a list
data = []
for file in datapath.glob("lab2*.csv"):

    ## Extract sample and team information from file name
    match = meta.search(file.name)

    ## Load data
    try:
        _df = tools.load_data(file, datastart=45)
        _df['team'] = match.group('team')
        data.append(_df)
    except AssertionError as e:
        print(f"Error loading {file.name}: {e}")

## Concatenate all data into a single dataframe
raw_data = (pd.concat(data)
    .sort_values(['team', 'time'])
    .reset_index(drop=True))

## Load specimen dimensions
specimen = pd.read_excel(datapath / 'lab2_specimen_dimensions.xlsx')

data = []

for team in raw_data['team'].unique():
    ## Extract data for this team
    _df = (raw_data
        .query(f"team == '{team}'")
        .reset_index(drop=True))

    ## Assign cycles
    _df = tools.assign_cycles(_df)

    ## Store data and show plot
    data.append(_df)

## Concatenate all data into a single dataframe
data = pd.concat(data).reset_index(drop=True)

# Compute strain and stress
for team in data['team'].unique():
    I = data['team'] == team
    J = specimen['Group #'] == int(team)

    # Stress
    A = (specimen.loc[J, 'Width (mm)'] * specimen.loc[J, 'Thickness (mm)']).values[0]
    data.loc[I, 'stress'] = data.loc[I, 'load'] / A

    # Strain
    L0 = specimen.loc[J, 'Gauge length (mm)'].values[0]
    data.loc[I, 'strain'] = data.loc[I, 'disp'] / L0

data = (data
    .query("(cycle == 1 or cycle == 10) and stage == 'loading'")
    .reset_index(drop=True))

display(data.head())

,time,disp,load,team,cycle,stage,stress,strain
0,0.0000,0.000000,0.05,01,1,loading,0.022727,0.000000
1,0.1464,0.000216,0.05,01,1,loading,0.022727,0.000025
2,0.2928,0.000520,0.05,01,1,loading,0.022727,0.000060
3,0.4392,0.007902,0.07,01,1,loading,0.031818,0.000908
4,0.5856,0.015485,0.07,01,1,loading,0.031818,0.001780


In [75]:
# Loop through teams
# Loop through cycles (1 and 10)
for team in data.team.unique():
    for cycle in [1, 10]:
        I = (data.team == team) & (data.cycle == cycle) & (data.stage == 'loading')

        _data = data.loc[I, ['strain', 'stress']].copy()

        lin_strain, lin_stress, E, r2 = tools.find_linear_region(_data, cutoff=0.5)
        
        fig = tools.plot_fit_stress_strain(_data, lin_stress, lin_strain)
        fig.update_layout(
            width=600, 
            height=400,
            title=f"Team {team} - Cycle {cycle}",
            title_x=0.5,
            title_y=0.8,
            template='plotly_white',
        );
        fig.show()